# apsis on the BRML cluster

Generally, apsis consists of a server, whose task it is to generate new candidates and receive updates, and several worker processes, who evaluate the actual machine learning algorithm and update the server.
Right now, it's best if you start the server on your own computer, and the worker processes as jobs on the cluster.

To start with, you need to install apsis and one requirement. To do so, first clone the apsis repo.
    
    git clone https://github.com/FrederikDiehl/apsis.git
    
And add it to the python path (or call sys.path.append(YOUR_PATH) everytime you need it).

Additionally, you'll need a newer requests version; locally at least.

    pip install --upgrade --user requests
    
Now, change to the cloned apsis directory, and change to the brml_dev branch

    git checkout brml_dev

I'll keep the current mostly-stable version with some hacks for brml there. You can then either start the server in a python shell, or with the REST_start_script in code/webservice. In the python shell (don't do this here, because it blocks the shell), do

    from apsis.webservice.REST_start_script import start_rest
    start_rest(port=5116)
    
Or whichever port you want to use. You can do the rest here, now.
But, first of all, try to access HOSTNAME:5116 via browser. You should see something like this: 

    {
      "result": "failed"
    }
    
Congratulations; that means the server is working.

Now, first of all, let's look at the experiments page. The site to access (also via browser) is simply `HOSTNAME:5116/experiments`, the result should look like this:

    {
      "result": []
    }
    
This means the result of our request (getting all experiment ids) was successful, but we have no experiments started. Let's change that!

In [3]:
from apsis_client.apsis_connection import Connection
conn = Connection(server_address="http://CN-2:5116")
#conn = Connection(server_address="http://pc-hiwi6:5116")

This is the Connection object, which we'll use to interface with the server. I've used `PC-HIWI6:5116` as my hostname (and yes, the `http` is important); yours will vary. We can do the same we've done before, and look for experiment ids.

In [4]:
conn.get_all_experiment_ids()

[]

Not surprisingly, there still aren't any experiments. Time to change that; let's build a simple experiment. We need to define several parameters for that:
* name: The human-readable name of the experiment.
* optimizer: The string defining the optimizer, can be either `RandomSearch` or `BayOpt`
* param_defs: The parameter definition dictionary, we'll come back to that in a bit.
* optimizer_arguments: The parameters for how the optimizer is supposed to work.
* minimization: Bool whether the problem is one of minimization or maximization.
Let's begin defining them.

In [5]:
name = "BraninHoo"
optimizer = "BayOpt"
minimization = True

Now, parameter definitions is interesting. It is a dictionary with string keys (the parameter names) and a dictionary defining the parameter. The latter dictionary contains the `type` field (defining the type of parameter definition). The other entries are the kwargs-like field to initialize the parameter definitions. 

For example, let's say we have two parameters. `x` is a numeric parameter between 0 and 10, and `class` is one of `"A"`, `"B"` or `"C"`. This, we define like this:
    
    param_defs = {
        "x": {
            "type": "MinMaxNumericParamDef",
            "lower_bound": 0,
            "upper_bound"; 10
        },
        "class": {
            "type": "NominalParamDef",
            "values": ["A", "B", "C"]
        }
    }
And that's it!

For our example, we'll use the BraninHoo function, so we need two parameters, called `x` and `y` (or, sometimes, called `x_0` and `x_1`, but that's ugly to type). `x` is between -5 and 10, `y` between 0 and 15.

In [6]:
param_defs = {
        "x": {
            "type": "MinMaxNumericParamDef", 
            "lower_bound": -5, 
            "upper_bound": 10
        },
        "y": {
            "type": "MinMaxNumericParamDef", 
            "lower_bound": 0, 
            "upper_bound": 15},
    }

We'll ignore optimizer_params for now. Usually, you could use it to set the number of samples initially evaluated via RandomSearch instead of BayOpt, or the optimizer used for the acquisition function, or the acquisition function etc.

Instead, we'll start with the initialization:

In [7]:
exp_id = conn.init_experiment(name, optimizer, 
                              param_defs, minimization=minimization,
                             blocking=True, timeout=10, )
                              #optimizer_arguments={"multiprocessing": "none"})
print(exp_id)

f04d498d91dc4ec4ab0ae916d1bcc12e


The experiment id is important for specifiying the experiment which you want to update, from which you want to get results etc. It can be set in `init_experiment`, but in doing so you have to be extremely careful not to use one already in use. If not specified, it's a newly generated `uuid4` hex, and is guarenteed not to occur multiple times.

Now, we had looked at all available experiment IDs before (when no experiment had been initialized). Let's do it again now.

In [8]:
conn.get_all_experiment_ids()

[u'f04d498d91dc4ec4ab0ae916d1bcc12e']

As you can see, the experiment now exists. Are there candidates already evaluated? Of course now, which the following can show us:

In [16]:
conn.get_all_candidates(exp_id)

{u'finished': [{u'cost': None,
   u'id': u'465b65caa98b4e5a993577704533ce2f',
   u'params': {u'x': 8.572377045489517, u'y': 5.655702685834349},
   u'result': 18.16465392594631,
   u'worker_information': None},
  {u'cost': None,
   u'id': u'b8c52178a37648eca0e2866e3a3d0aa2',
   u'params': {u'x': -0.9746260851257036, u'y': 0.6496615925539462},
   u'result': 64.73097851994237,
   u'worker_information': None},
  {u'cost': None,
   u'id': u'fb7b810cc81d421188aa5c6c3f416148',
   u'params': {u'x': 3.4701007781911937, u'y': 11.12570707015112},
   u'result': 83.5932096276205,
   u'worker_information': None},
  {u'cost': None,
   u'id': u'b54c33fe45a74a0aaf88fec29bbe2934',
   u'params': {u'x': -1.914426542197952, u'y': 13.65801308294684},
   u'result': 23.885073134980047,
   u'worker_information': None},
  {u'cost': None,
   u'id': u'1de7380c0d7d4c34bbddb685be880e50',
   u'params': {u'x': -4.773473000052572, u'y': 5.151492464204553},
   u'result': 140.30300297749193,
   u'worker_information': No

This function shows us three lists of candidates (currently empty). `finished` are all candidates that have been evaluated and are, well, finished. `pending` are candidates which have been generated, have possibly begun evaluating and then been paused. `working` are candidates currently in progress.

How do we get candidates? Simple, via the `get_next_candidate` function:

In [17]:
cand = conn.get_next_candidate(exp_id)
print(cand)

{u'cost': None, u'params': {u'y': 0.13325249050574528, u'x': 9.51675049160615}, u'id': u'c8d89a859f5b440c82d258558fcd28ae', u'worker_information': None, u'result': None}


A candidate is nothing but a dictionary with the following fields:
* cost: The cost of evaluating this candidate. Is currently unused, but can be used for statistics or - later - for Expected Improvement Per Second.
* params: The parameter dictionary. This contains one entry for each parameter, with each value being the parameter value for this candidate.
* id: The id of the candidate. Not really important for you.
* worker_information: This can be used to specify continuation information, for example. It will never be changed by apsis.
* result: The interesting field. The result of your evaluation.

Now it's time for your work; for evaluating the parameters. Here, let's use the BraninHoo function.

In [18]:
import math
def branin_func(x, y, a=1, b=5.1/(4*math.pi**2), c=5/math.pi, r=6, s=10,
                t=1/(8*math.pi)):
        # see http://www.sfu.ca/~ssurjano/branin.html.
        result = a*(y-b*x**2+c*x-r)**2 + s*(1-t)*math.cos(x)+s
        return result

And let's extract the parameters. Depending on your evaluation function, you can also just use the `param` entry dictionary directly (for example for sklearn functions).

In [19]:
x = cand["params"]["x"]
y = cand["params"]["y"]
result = branin_func(x, y)
print(result)

6.29690869373


Now, we'll just update the candidate with the result, and update the server:

In [20]:
cand["result"] = result
conn.update(exp_id, cand, status="finished")

u'success'

And let's look at the candidates again:

In [14]:
conn.get_all_candidates(exp_id)

{u'finished': [{u'cost': None,
   u'id': u'465b65caa98b4e5a993577704533ce2f',
   u'params': {u'x': 8.572377045489517, u'y': 5.655702685834349},
   u'result': 18.16465392594631,
   u'worker_information': None}],
 u'pending': [],
 u'working': []}

Yay, it worked!
And that's basically it. Every worker only has to use a few of the lines above (initializing the connection, getting the next candidate, evaluating and update).

In [15]:
import time

for j in range(5):
    candidates = []
    for i in range(5):
        candidates.append(conn.get_next_candidate(exp_id))
        print(i)
    for i in range(len(candidates)):
        x = candidates[i]["params"]["x"]
        y = candidates[i]["params"]["y"]
        result = branin_func(x, y)
        candidates[i]["result"] = result
        conn.update(exp_id, candidates[i], status="finished")
        print(i)
    print("Finished %i" %j)

0
1
2
3
4
0
1
2
3
4
Finished 0
0
1
2
3
4
0
1
2
3
4
Finished 1
0
1
2
3
4
0
1
2
3
4
Finished 2
0
1
2
3
4
0
1
2
3
4
Finished 3
0
1
2
3
4
0
1
2
3
4
Finished 4


In [ ]:
for cand in conn.get_all_candidates(exp_id)["finished"]:
    print(cand)